# <center>Forecasting Beer Demand using Seasonal ARIMA model </center>


### About this Notebook
In this notebook, I will be implementing the Seasonal ARIMA model on a non stationary time series data. The dataset can be downloaded using this [link](https://www.kaggle.com/sergiomora823/monthly-beer-production).

For a quick overview on the concept of Seasonal ARIMA model watch [this](https://youtu.be/WjeGUs6mzXg) video

### Importing Required packages

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Lets read in the data
df = pd.read_csv("../input/monthly-beer-production/datasets_56102_107707_monthly-beer-production-in-austr.csv")
df.head()

As we can see, it is a very simple dataframe with monthly beer production values from January 1956

In [ ]:
#lets look at the brief summary of the Dataframe
df.info()


In [ ]:
df['Month']=pd.to_datetime(df['Month'])

In [ ]:
df.dtypes

In [ ]:
# Lets set the month column as the index for our Dataframe
df.set_index('Month',inplace=True)

In [ ]:
# Basic plot to get the general idea of the trends in data
df.plot()

In [ ]:
df.iloc[:60].plot()

#### When we look closely into our data,we can clearly see that there is some annual seasonality

### Selecting appropriate Model:
To continue with out time series analysis, we must first test for stationarity
For this, let's use Dickey–Fuller test 

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
test=adfuller(df['Monthly beer production'])
print (test)


## Note:
Running adfuller on the beer production column returns the following values in order : (ADF Test Statistic, p-value, No. of Lags Used, No. of Observations Used,{Critical Values},icbest)

The adfuller test returned a p-value of 0.178. We can assume that the data is stationary only if the p-value is very low (less than 0.05). To acheive stationarity, lets difference the production data by twelve months and check the p-value using the adfuller test

In [ ]:
annual_difference = df['Monthly beer production'] - df['Monthly beer production'].shift(12)
adfuller(annual_difference.dropna())

As suspected,some annual seasonality is present since the p-value has reduced below 0.05 for 12 months difference. Hence we can assume stationarity when differencing by 12 months

In [ ]:
df['Annual Difference']=annual_difference

In [ ]:
(df['Annual Difference'].iloc[12:48]).plot()

In [ ]:
df.head()

### Since the data is non stationary, SARIMA model will be suitable to predict this data
#### Seasonal ARIMA MODEL  (Auto Regressive Integrated Moving Average) :
* AutoRegression(p) : A model that accounts for the relationship between an observation and n number of lagged observations
* Integrated(d) : The use of differencing of observations to achieve stationarity
* Moving Average(q) : A model that accounts for the linear dependancy of output variable on the current and various past values

#### For applying the ARIMA model we need p,q and q values:
* p is the AR model lags and can be determined from the pacf plot
* d is the degree of differencing
* q is the size of the moving average window (order)

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['Annual Difference'].iloc[13:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['Annual Difference'].iloc[13:],lags=40,ax=ax2)

* For an AR model, the theoretical PACF “shuts off” past the order(p) of the model
* For an MA model, the theoretical PACF does not shut off, but instead tapers toward 0 in some manner. A clearer pattern for an   MA model is in the ACF. The ACF will have non-zero autocorrelations only at lags involved in the model.

Since our data is not an ideal case of the ARIMA model, the trend is not clearly visible. 
We will take p,d,q values as (1,1,1) and seasonal factor as 12 (for SARIMA model)
To learn more on selecting p,d,q values from the autocorrelation charts refer to this [paper](https://online.stat.psu.edu/stat510/book/export/html/662)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
x_test=df['Monthly beer production'].iloc[:400]

In [ ]:
# Create Model with first 400 values and use it to predict the remaining values inorder to check accuracy
model=ARIMA(x_test,order=(1,1,1))
model_fit=model.fit()

In [ ]:
df['ARIMA forecast']=model_fit.predict(start=400,end=470,dynamic=True)
df[['Monthly beer production','ARIMA forecast']].plot(figsize=(12,8))

As expected the ARIMA model did not return a good prediction since the model does not account for seasonality. So lets implement Seasonal ARIMA model

In [ ]:
model=sm.tsa.statespace.SARIMAX(x_test,order=(1, 1, 1),seasonal_order=(1,1,1,12))
results=model.fit()

In [ ]:
df['SARIMA forecast']=results.predict(start=400,end=470,dynamic=True)
df[['Monthly beer production','SARIMA forecast']].plot(figsize=(12,8))

In [ ]:
df[['Monthly beer production','SARIMA forecast']].iloc[301:470].plot(figsize=(12,8))

From visual analysis, we can see that the SARIMA model performed reasonably well. However, to understand the accuracy of the model mathematically, we can calculate the residual and find the mean absolute percent error and root mean squared error.

In [ ]:
residuals=df['Monthly beer production']-df['SARIMA forecast']
residuals.dropna(inplace=True)

In [ ]:
print('Mean Absolute Percent Error:', round(np.mean(abs(residuals/df['Monthly beer production'].iloc[400:471])),4))
print('Root Mean Squared Error:', np.sqrt(np.mean(residuals**2)))

In [ ]:
residuals.plot()

Now, lets forecast values for the next two years

In [ ]:
from pandas.tseries.offsets import DateOffset
future_dates=[df.index[-1]+ DateOffset(months=x)for x in range(0,24)]

In [ ]:
future_datest_df=pd.DataFrame(index=future_dates[1:],columns=df.columns)

In [ ]:
future_df=pd.concat([df,future_datest_df])

In [ ]:
future_df['futureforecast'] = results.predict(start = 474, end = 500, dynamic= True)  
future_df[['Monthly beer production', 'futureforecast']].plot(figsize=(12, 8))

In [ ]:
future_df[['Monthly beer production', 'futureforecast']].iloc[400:].plot(figsize=(8, 8))